In [1]:
architecture_config = [  # kernel_size in_channel out_channel stride padding
    (7, 64, 2, 3),
    "M",
    (3, 192, 1, 1),
    "M",
    (1, 128, 1, 0),
    (3, 256, 1, 1),
    (1, 256, 1, 0),
    (3, 512, 1, 1),
    "M",
    [(1, 256, 1, 0), (3, 512, 1, 1), 4],  # 4 is number of times to repeat
    (1, 512, 1, 0),
    (3, 1024, 1, 1),
    "M",
    [(1, 512, 1, 0), (3, 1024, 1, 1), 2],  # 2 is number of times to repeat
    (3, 1024, 1, 1),
    (3, 1024, 2, 1),
    (3, 1024, 1, 1),
    (3, 1024, 1, 1),
]

In [2]:
import os
import import_ipynb
import torch
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision.transforms.functional as FT
from tqdm import tqdm
from torch.utils.data import DataLoader
from model import Yolov1
from dataloader import VOCDataset
from utils import (
    non_max_suppression,
    mean_average_precision,
    intersection_over_union,
    cellboxes_to_boxes,
    get_bboxes,
    plot_image,
    save_checkpoint,
    load_checkpoint,
)
from loss import YoloLoss

seed = 123
torch.manual_seed(seed)

# Hyperparameters etc. 
LEARNING_RATE = 2e-5
DEVICE = "cuda" if torch.cuda.is_available else "cpu"
BATCH_SIZE = 8 # 64 in original paper but I don't have that much vram, grad accum?
WEIGHT_DECAY = 0
EPOCHS = 100
NUM_WORKERS = 6
PIN_MEMORY = True
LOAD_MODEL = False
LOAD_MODEL_FILE = "overfit.pth.tar"



class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, bboxes):
        for t in self.transforms:
            img, bboxes = t(img), bboxes

        return img, bboxes


transform = Compose([transforms.Resize((448, 448)), transforms.ToTensor(),])


def train_fn(train_loader, model, optimizer, loss_fn):
    loop = tqdm(train_loader, leave=True)
    mean_loss = []

    for batch_idx, (x, y) in enumerate(loop):
        x, y = x.to(DEVICE), y.to(DEVICE)
        out = model(x)
        loss = loss_fn(out, y)
        mean_loss.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # update progress bar
        loop.set_postfix(loss=loss.item())

    print(f"Mean loss was {sum(mean_loss)/len(mean_loss)}")


def main():
    model = Yolov1(architecture_config).to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY
    )
    loss_fn = YoloLoss()

    if LOAD_MODEL:
        load_checkpoint(torch.load(LOAD_MODEL_FILE), model, optimizer)

    train_dataset = VOCDataset(
        csv_file = "/home/mskang/hyeokjong/object_detection/voc_2007/archive/train.csv",
        transform=transform,
        img_dir='/home/mskang/hyeokjong/object_detection/voc_2007/archive/images',
        label_dir='/home/mskang/hyeokjong/object_detection/voc_2007/archive/labels',
    )

    test_dataset = VOCDataset(
        csv_file = "/home/mskang/hyeokjong/object_detection/voc_2007/archive/test.csv",
        transform=transform,
        img_dir='/home/mskang/hyeokjong/object_detection/voc_2007/archive/images',
        label_dir='/home/mskang/hyeokjong/object_detection/voc_2007/archive/labels',
    )

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        shuffle=True,
        drop_last=True,
    )

    for epoch in range(EPOCHS):
         for x, y in train_loader:
            x = x.to(DEVICE)
            for idx in range(8):
                bboxes = cellboxes_to_boxes(model(x))
                bboxes = non_max_suppression(bboxes[idx], iou_threshold=0.5, threshold=0.4, box_format="midpoint")
                plot_image(x[idx].permute(1,2,0).to("cpu"), bboxes)

            import sys
            sys.exit()

        pred_boxes, target_boxes = get_bboxes(
            train_loader, model, iou_threshold=0.5, threshold=0.4
        )

        mean_avg_prec = mean_average_precision(
            pred_boxes, target_boxes, iou_threshold=0.5, box_format="midpoint"
        )
        print(f"Train mAP: {mean_avg_prec}")

        #if mean_avg_prec > 0.9:
        #    checkpoint = {
        #        "state_dict": model.state_dict(),
        #        "optimizer": optimizer.state_dict(),
        #    }
        #    save_checkpoint(checkpoint, filename=LOAD_MODEL_FILE)
        #    import time
        #    time.sleep(10)

        train_fn(train_loader, model, optimizer, loss_fn)


if __name__ == "__main__":
    main()

importing Jupyter notebook from model.ipynb
torch.Size([32, 64, 224, 224])
0 (7, 64, 2, 3) <class 'tuple'>
1 M <class 'str'>
2 (3, 192, 1, 1) <class 'tuple'>
3 M <class 'str'>
4 (1, 128, 1, 0) <class 'tuple'>
5 (3, 256, 1, 1) <class 'tuple'>
6 (1, 256, 1, 0) <class 'tuple'>
7 (3, 512, 1, 1) <class 'tuple'>
8 M <class 'str'>
9 [(1, 256, 1, 0), (3, 512, 1, 1), 4] <class 'list'>
10 (1, 512, 1, 0) <class 'tuple'>
11 (3, 1024, 1, 1) <class 'tuple'>
12 M <class 'str'>
13 [(1, 512, 1, 0), (3, 1024, 1, 1), 2] <class 'list'>
14 (3, 1024, 1, 1) <class 'tuple'>
15 (3, 1024, 2, 1) <class 'tuple'>
16 (3, 1024, 1, 1) <class 'tuple'>
17 (3, 1024, 1, 1) <class 'tuple'>
torch.Size([2, 1470])
importing Jupyter notebook from dataloader.ipynb
importing Jupyter notebook from utils.ipynb
importing Jupyter notebook from loss.ipynb
torch.Size([32, 7, 7, 2])
Train mAP: 0.0


  0%|          | 3/2068 [00:01<19:07,  1.80it/s, loss=541]


RuntimeError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 11.78 GiB total capacity; 2.72 GiB already allocated; 86.69 MiB free; 2.86 GiB reserved in total by PyTorch)